<a href="https://colab.research.google.com/github/nolan8060/nolan8060/blob/main/Homework_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Homework 4, due Thursday, February 5**

# Problems 1 and 2

These problems are to be done **by hand** and submitted separately from Problems 3 and 4. Please submit an image or pdf of your solutions to Gradescope.

1. **Winston, p. 844, Problem 2.** Solve the LP by hand.

2. A variation of rock-paper-scissors is as follows. There are two players. First, each player throws out both hands, each of which is rock, paper, or scissors. The hands can be the same or different. Then, after seeing what the opponent has thrown, they simultaneously choose one of their own hands to put away. The game then resolves as usual with the hands that are remaining. Both players are trying to maximize the difference between theire score and their opponent's.

    a. Suppose that P1 plays rock-paper and P2 plays paper-scissors. Assuming they both play optimally from this point on, what are the optimal strategies for each player, and what is the value of the game? Solve the LP by hand.

    b. Do the same for the situation where both players play rock-paper.

    c. Explain why during the first phase of the game, there is no reason for a player to play two of the same hand (e.g. rock-rock).


# Problem 3

In this problem we will write a program to solve a general two-player constant sum game.

The input is
*   The list of strategies for P1
*   The list of strategies for P2
*   The payoff matrix for P1, as a dictionary whose keys are pairs (P1 strategy, P2 strategy), and the value for this key is the expected payoff for P1 if those strategies are played.

The output is
* The value of the game
* An optimal mixed strategy for P1, as a dictionary whose key-value pairs are {strategy : probability of playing that strategy}
* An optimal mixed strategy for P2, same.



In [14]:
%pip install ortools

Complete the following code.

In [15]:
from ortools.linear_solver import pywraplp
import math

def solve_game(p1_strats, p2_strats, payoff):

  # Create the linear program solver for P1's optimal strategy.
  solver = pywraplp.Solver.CreateSolver("GLOP")

  # Create the variables for the first LP.
  z = solver.NumVar(-solver.infinity(), solver.infinity(), 'z')
  x = {}
  for strat in p1_strats:
    x[strat] = solver.NumVar(0, 1, "")

  # Set the objective.
  solver.Maximize(z)

  # Create the constraints. Use payoff[(strat1,strat2)] to access the payoff if
  # player 1 plays strat1, player 2 plays strat2.
  solver.Add(sum(x[strat] for strat in p1_strats) == 1)
  for strat2 in p2_strats:
    solver.Add(sum(payoff[(strat1, strat2)] * x[strat1] for strat1 in p1_strats) >= z)

  # Solve and store the solution. DO NOT CHANGE.
  status = solver.Solve()
  if status == pywraplp.Solver.OPTIMAL:
    value1 = solver.Objective().Value()
    optimalstrat1 = {strat:x[strat].solution_value() for strat in p1_strats}
  else:
    return "Error in solving first LP", None, None


  # Create the linear program solver for P2's optimal strategy.
  solver = pywraplp.Solver.CreateSolver("GLOP")

  # Create the variables for the second LP.
  z = solver.NumVar(-solver.infinity(), solver.infinity(), 'z')
  y = {}
  for strat in p2_strats:
    y[strat] = solver.NumVar(0, 1, "")


  # Set the objective.
  solver.Minimize(z)


  # Create the constraints
  solver.Add(sum(y[strat] for strat in p2_strats) == 1)
  for strat1 in p1_strats:
    solver.Add(sum(payoff[(strat1, strat2)] * y[strat2] for strat2 in p2_strats) <= z)


  # Solve and store the solution. DO NOT CHANGE
  status = solver.Solve()
  if status == pywraplp.Solver.OPTIMAL:
    value2 = solver.Objective().Value()
    optimalstrat2 = {strat:y[strat].solution_value() for strat in p2_strats}
  else:
    return "Error in solving second LP", None, None

  # Checks that the optimal values of the two linear programs are the same.
  # If not, then something is wrong with your code.
  if not math.isclose(value1, value2, abs_tol=0.0001):
    return "Error: values not same", None, None

  return value1, optimalstrat1, optimalstrat2

Test your code on the penality kicks problem we did in class below.

In [16]:
kicker_strats = ["kick left", "kick right"]
goalie_strats = ["lean left", "lean right"]

# Complete this
payoff = {}
payoff[("kick left", "lean left")] = 0
payoff[("kick left", "lean right")] = 0.8
payoff[("kick right", "lean right")] = 0
payoff[("kick right", "lean left")] = 0.4

value, optstrat1, optstrat2 = solve_game(kicker_strats, goalie_strats, payoff)
print(value)
print(optstrat1)
print(optstrat2)

0.26666666666666666
{'kick left': 0.33333333333333337, 'kick right': 0.6666666666666666}
{'lean left': 0.6666666666666667, 'lean right': 0.3333333333333333}


In addition, run the following code to check with **Winston, p. 844, Problem 3b**. Check that your answer matches the one in the back of the book on page 1392.

In [17]:
p1_strats = [1,2,3]
p2_strats = [1,2,3]

a = [[1/2,-1,-1],
     [-1,1/2,-1],
     [-1,-1,1]]

payoff = {(i,j):a[i-1][j-1] for i in p1_strats for j in p2_strats}

value, optstrat1, optstrat2 = solve_game(p1_strats, p2_strats, payoff)
print(value)
print(optstrat1)
print(optstrat2)

-0.4545454545454545
{1: 0.36363636363636365, 2: 0.36363636363636354, 3: 0.27272727272727276}
{1: 0.36363636363636365, 2: 0.36363636363636354, 3: 0.27272727272727276}


# Problem 4

We will now find the optimal strategy for the first phase of the game in Problem 2. (If you think about it, the answer should be intuitively clear, but we will verify it with a linear program.) Complete the following code to determine an optimal strategy for both players for the phase where they throw two hands. Then answer the follow-up question.

Hint: Using Problem 2, you can eliminate some strategies, greatly reducing the number of cases you have to deal with. You can then use Problem 2 to fill in the rest of the payoff table. If you carefully think about symmetries, you can do this using only the answer to Problem 2 without having to solve additional linear programs.

In [18]:
p1_strats = ["RP","RS","PS"] # Complete this
p2_strats = ["RP","RS","PS"] # Complete this

payoff = {}
payoff[("RP","RP")] = 0 # Complete this
payoff[("RS","RS")] = 0
payoff[("PS","PS")] = 0
payoff[("RP","PS")] = -1/3
payoff[("PS","RP")] = 1/3
payoff[("RP","RS")] = 1/3
payoff[("RS","RP")] = -1/3
payoff[("RS","PS")] = 1/3
payoff[("PS","RS")] = -1/3


# Do not change anything past this point.
value, optstrat1, optstrat2 = solve_game(p1_strats, p2_strats, payoff)
print(value)
print(optstrat1)
print(optstrat2)

2.435226118817806e-17
{'RP': 0.33333333333333337, 'RS': 0.3333333333333333, 'PS': 0.33333333333333337}
{'RP': 0.33333333333333337, 'RS': 0.3333333333333333, 'PS': 0.33333333333333337}


**Explain below** how you know the value of this game without having to solve the LP. If this doesn't exactly match what you got in the LP, explain why.

**Explanation**: It's almost the same as playing the same rock paper scissors. We already solved in problem 2 that in a game with non-identical hands the best payoff will be 1/3 and -1/3. There would be no difference for the payoff for the other non-identical hands.

**Submission:** As usual, make sure you click "Run all", and all output is shown. Submit Problems 1+2 and Problems 3+4 separately on Gradescope.